In [3]:
from os import listdir
from os.path import isfile, join
import pickle
from IPython.display import clear_output

%run ./generator.ipynb
%run ./variables.ipynb
%run ./utils.ipynb

In [4]:
images = [f for f in listdir(DATASET_PATH) if isfile(join(DATASET_PATH, f))]

tmp_code = {}
i = 1
for file in images:
    taxon = file.split('_')[1]
    if not (taxon in tmp_code):
        tmp_code[taxon] = i
        i+=1

savePickle(tmp_code, SAVE_PATH+"/maps/multiclass_label_map.pickle")
binary_tmp_code = {}
binary_tmp_code["diatom"] = 1
savePickle(binary_tmp_code, SAVE_PATH+"/maps/binary_label_map.pickle")

In [9]:
n_id = 0
print("Generating", N_IMAGES, "images!")
for i in range(N_IMAGES):
    string_id = '{:04d}'.format(n_id)
    final_img, labels, individual_masks = Generator_Img(DATASET_PATH = DATASET_PATH, simple_angles=False, size_px=1000, fast=True, verbose=False)
    saveImg(final_img, SAVE_PATH+"/images/"+string_id+".png");
    savePickle(labels, SAVE_PATH+"/annotations/bb_"+string_id+".pickle")
    ## Saving individual masks
    taxon_n = {}
    paths = []
    for mask in individual_masks:
        taxon = mask["taxon"]
        if taxon in taxon_n:
            taxon_n[taxon] += 1
        else:
            taxon_n[taxon] = 0
        path_mask = "masks/"+string_id+"_"+taxon+"_"+'{:03d}'.format(taxon_n[taxon])+".png"
        paths.append(path_mask)
        # Saving mask
        img = PIL.Image.fromarray(mask["patch_mask"])
        #output = io.BytesIO()
        check_dirs(join(SAVE_PATH, path_mask))
        img.save(join(SAVE_PATH, path_mask), format='PNG')
        #encoded_mask_png_list.append(output.getvalue())
        #saveImg(mask["patch_mask"], join(SAVE_PATH, path_mask));
    savePickle(paths, SAVE_PATH+"/annotations/masks_"+string_id+".pickle")
    clear_output(wait=True)
    print(n_id+1,"/",N_IMAGES)
    n_id += 1

16 / 200


KeyboardInterrupt: 